In [28]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from splinter import browser
from selenium import webdriver
import urllib.request

## NASA MARS NEWS

In [29]:
url = "https://mars.nasa.gov/news"

In [30]:
# Retrieve page with the requests module
#response = requests.get(url)

In [31]:
#soup = bs(response.text, 'html.parser')

In [32]:
#print(soup.prettify())

In [33]:
#news_title = soup.find("div", class_ = "content_title").text
#news_title

In [34]:
driver = webdriver.Firefox()
driver.get(url)
html = driver.page_source
#html = browser.html

In [35]:
# Scrape the webpage using beautiful soup
soup = bs(html, 'html.parser')

In [36]:
#print(soup.prettify())

In [37]:
# Latest title and remov the new line 
news_titles = soup.find('div', class_ = 'content_title').get_text(strip=True)
news_titles

'The MarCO Mission Comes to an End'

In [38]:
# Convert all the titles into a dict
# titles_dict = [
#    {
#        "title": news_title,
#        #"url": urllib.parse.urljoin("https://www.reddit.com/", a_tag["href"]),
#    }
#    for news_title in news_titles
# ]

In [39]:
#titles_dict

In [40]:
# Latest Paragraph
news_paragraph = soup.find('div', class_ = 'article_teaser_body').get_text()

In [41]:
news_paragraph

'The pair of briefcase-sized satellites made history when they sailed past Mars in 2019.'

## JPL Mars Space Images - Featured Image

In [42]:
image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"

In [43]:
# Retrieve page with the requests module
response = requests.get(image_url)

In [44]:
soup = bs(response.text, 'html.parser')

In [45]:
featured_url = soup.find("a", class_="button fancybox")["data-link"]

In [46]:
featured_url

'/spaceimages/details.php?id=PIA16711'

In [47]:
featured_image_url = f'https://www.jpl.nasa.gov{featured_url}'
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/details.php?id=PIA16711'

## Mars Weather

In [48]:
weather_url = "https://twitter.com/marswxreport?lang=en"

In [49]:
response = requests.get(weather_url)

In [50]:
driver = webdriver.Firefox()
driver.get(weather_url)
html = driver.page_source

In [51]:
soup = bs(response.text, 'html.parser')

In [52]:
# Extract title text
mars_weather = soup.find(class_='tweet-text').text
mars_weather

'InSight sol 445 (2020-02-26) low -92.8ºC (-135.0ºF) high -12.8ºC (8.9ºF)\nwinds from the SSE at 5.9 m/s (13.3 mph) gusting to 21.1 m/s (47.3 mph)\npressure at 6.30 hPapic.twitter.com/ihFGVkib6L'

## Mars Facts

In [53]:
import pandas as pd
facts_url = "http://space-facts.com/mars/"

In [54]:
mars_data = pd.read_html(facts_url)

ImportError: lxml not found, please install it

## Mars Hemispheres

In [ ]:
hemisphere_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

In [ ]:
response = requests.get(hemisphere_url)

In [ ]:
soup = bs(response.text, 'html.parser')

In [ ]:
hemi_img_url_list = []
title = []

In [ ]:
for item in soup.select("a.itemLink"):
    title.append(item.select("h3")[0].get_text())
    url_new = "https://astrogeology.usgs.gov"+item["href"]
    response_new = requests.get(url_new)
    soup_new = bs(response_new.text, 'html.parser')
    hemi_img_url_list.append("https://astrogeology.usgs.gov"+soup_new.select('img.wide-image')[0]['src'])
    

In [ ]:
hemisphere_image_url = [{"Title":title[i],'img_url':hemi_img_url_list[i]} for i in range(len(title))]
hemisphere_image_url